In [2]:
import numpy as np, pandas as pd
from keras.preprocessing import image
import tensorflow as tf
import os

2023-05-28 06:49:54.975602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-28 06:49:55.315072: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-28 06:49:55.315093: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-28 06:49:55.358456: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-28 06:49:56.526207: W tensorflow/stream_executor/platform/de

In [3]:
root = '/home/csantiago3/code/carlajord/week_5/baseline/methane_224_data' #os.getcwd()
data = pd.read_csv('methane_224_data/smallsize224_all.csv')
data.head(2)

,Unnamed: 0,Latitude,Longitude,Type,Source,Image_Folder,geometry,dataset,Image_Folder_long_lat,img_dir
0,0,43.948271,-93.649496,CAFOs,MinnesotaPollutionControlAgency,train_images/43.94827109_-93.6494963,POLYGON ((-93.64626236497716 43.95060750304138...,train,43.94827109_-93.6494963,F:\\CNOOC_testing\\Methane_dataset\\METHANE_PR...
1,1,43.004434,-78.208900,Negative,Aeroplanes,train_images/43.00443446102501_-78.20890003579709,POLYGON ((-78.20566610077425 43.00680791855935...,train,43.00443446102501_-78.20890003579709,F:\\CNOOC_testing\\Methane_dataset\\METHANE_PR...


In [4]:
data['new_path'] = data['img_dir'].apply(lambda x: x.replace("F:\\\\CNOOC_testing\\\\Methane_dataset\\\\METHANE_PROJECT\\\\", "") \
         .replace("\\\\", "/").replace("\\", "/"))
os.path.exists(os.path.join(root, data['new_path'].iloc[0]))

True

In [5]:
data.shape

(86599, 11)

In [6]:
# drop unknowns
data = data.dropna()

# leave just unique categories
data = data[(data.Type == 'Negative') | (data.Type == 'CAFOs') | (data.Type == 'WWTreatment')
           | (data.Type == 'Landfills') | (data.Type == 'RefineriesAndTerminals')
           | (data.Type == 'ProcessingPlants') | (data.Type == 'Mines')]

# shuffle in-place
data = data.sample(frac=1).reset_index(drop=True)


In [7]:
data.shape

(86188, 11)

In [8]:
df_train = data.loc[data['dataset'] == "train"]
df_test = data.loc[data['dataset'] == "val"]

# reset indices
df_train.reset_index(inplace=True, drop=True)
df_test.reset_index(inplace=True, drop=True)

In [9]:
df_train.shape, df_test.shape

((84693, 11), (505, 11))

# Baseline model

In [12]:
SEED = 42

IMG_HEIGHT = 224
IMG_WIDTH = 224

BASELINE_SPLIT = 0.2 # Run baseline with 10% of the training data

In [13]:
base_data, _ = np.split(df_train, [int(BASELINE_SPLIT*len(df_train))])
#base_data = df_train.sample(n=BASELINE_BATCH)

In [14]:
dict(df_train.Type.value_counts()/len(df_train))

{'Negative': 0.40375237622944044,
 'CAFOs': 0.2944044962393586,
 'WWTreatment': 0.16879789356853578,
 'Landfills': 0.046225780170734296,
 'RefineriesAndTerminals': 0.04506865974755883,
 'ProcessingPlants': 0.02160745280011335,
 'Mines': 0.020143341244258674}

In [15]:
proba_predictions = dict(base_data.Type.value_counts()/len(base_data))
proba_predictions

{'Negative': 0.4009918526390365,
 'CAFOs': 0.29596174282678,
 'WWTreatment': 0.16961860904475146,
 'Landfills': 0.04669972842130122,
 'RefineriesAndTerminals': 0.04622741764080765,
 'ProcessingPlants': 0.020545518951470068,
 'Mines': 0.019955130475853112}

In [16]:
BASELINE_BATCH = base_data.shape[0]

In [17]:
class MyDataFrameIterator(image.DataFrameIterator):
    
    def __getitem__(self, idx):
        X_, y_ = super().__getitem__(idx)
        y_ = np.delete(y_, self.class_indices['Negative'], axis=1)
        return X_, y_
    
    def next(self):
        X_, y_ = super().next()
        y_ = np.delete(y_, self.class_indices['Negative'], axis=1)
        return X_, y_
    
class MyImageDataGenerator(image.ImageDataGenerator):
    
    def flow_from_dataframe(self, df, directory, *args, **kwargs):
        return MyDataFrameIterator(df, directory, *args, **kwargs)

In [24]:
# Instantiate metrics
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
auc = tf.keras.metrics.AUC()
pr_auc = tf.keras.metrics.AUC(name='pr_auc', curve='PR')
accuracy = tf.keras.metrics.Accuracy()

metrics_fun = {'Precision': precision,
               'Recall': recall,
               'AUC': auc,
               'PR-AUC': pr_auc,
               'Accuracy': accuracy}

2023-05-28 07:17:03.075942: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-28 07:17:03.075992: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-28 07:17:03.076012: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (SLB-980BVT2): /proc/driver/nvidia/version does not exist
2023-05-28 07:17:03.076216: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [53]:
def compute_metrics(y_true, y_pred, metrics_fun, cat_idx):
    
    """Computes metrics and stores results in list"""

    metrics_results = []
    for func in metrics_fun:
        func.reset_states()
        metrics_results.append(func(y_true[:][:,cat_idx].reshape(BASELINE_BATCH,1),
                                    y_pred[:][:,cat_idx].reshape(BASELINE_BATCH,1)).numpy())
    return metrics_results

In [18]:
# Get y_true
datagen = MyImageDataGenerator(rescale=1./255)
train_gen = datagen.flow_from_dataframe(
                    base_data,
                    root,
                    x_col="new_path",
                    y_col="Type",
                    color_mode='rgb',
                    seed = SEED,
                    class_mode="categorical",
                    validate_filenames=False,
                    target_size=(IMG_HEIGHT,IMG_WIDTH),
                    batch_size=BASELINE_BATCH)
_, y_true = train_gen.next()

Found 16938 non-validated image filenames belonging to 7 classes.


In [21]:
cat_names = list(train_gen.class_indices.keys())
cat_names.remove('Negative')
probs = {}
for cat in cat_names:
    probs[cat] = proba_predictions[cat]
probs

{'CAFOs': 0.29596174282678,
 'Landfills': 0.04669972842130122,
 'Mines': 0.019955130475853112,
 'ProcessingPlants': 0.020545518951470068,
 'RefineriesAndTerminals': 0.04622741764080765,
 'WWTreatment': 0.16961860904475146}

In [32]:
# generate Y pred
def compute_baseline_metric(y_true, prob, metrics_fun):

    """ Generates y prediction for given category probability """
    
    # instantiate y_pred
    y_pred = np.zeros_like(y_true)
    # get indices for values to be assigned True
    n_samples = y_pred.shape[0]
    indices = np.random.choice(n_samples, np.round(n_samples*prob).astype(np.uint32), replace=False)
    y_pred[indices] = 1

    # compute metrics
    metrics_results = []
    for func in metrics_fun:
        func.reset_states()
        metrics_results.append(func(y_true.reshape(BASELINE_BATCH, 1),
                                    y_pred.reshape(BASELINE_BATCH,1)).numpy())
    return metrics_results



In [57]:
cat_metrics = {}
for cat_idx, cat in enumerate(list(probs.keys())):
    
    # Instantiate dictionary that stores metrics per category
    cat_metrics[cat] = {}
    for m in metrics_fun.keys():
        cat_metrics[cat][m] = []

    # Take 5 samples of y_prediction for each category and compute average metrics
    cnt = 0
    while cnt < 2:

        m_results = compute_baseline_metric(y_true[:, cat_idx], probs[cat], metrics_fun.values())
        
        # Store all calculated metrics
        for m_idx, m in enumerate(metrics_fun.keys()):
            cat_metrics[cat][m].append(m_results[m_idx])
        cnt+=1
        
    for m in metrics_fun.keys():
        cat_metrics[cat][m] = np.mean(np.array(cat_metrics[cat][m]))
    

In [58]:
pd.DataFrame(cat_metrics)

,CAFOs,Landfills,Mines,ProcessingPlants,RefineriesAndTerminals,WWTreatment
Precision,0.295332,0.044880,0.011834,0.015805,0.040230,0.164636
Recall,0.295332,0.044880,0.011834,0.015805,0.040230,0.164636
AUC,0.499553,0.499046,0.495857,0.497580,0.496856,0.497000
PR-AUC,0.295699,0.046447,0.019378,0.020182,0.045375,0.167833
Accuracy,0.582891,0.910792,0.960562,0.959558,0.911265,0.716614
